# Firehose

Basic implementation via websocket

In [ ]:
const firehoseWS = () => {
  try {
    const ws = new WebSocket(
      "wss://jetstream2.us-east.bsky.network/subscribe?wantedCollections=xyz.statusphere.status",
    );

    ws.onopen = () => {
      console.log("Connected to Bluesky WebSocket");
    };

    ws.onmessage = (event) => {
      try {
        const data = JSON.parse(event.data);

        if (data.kind === "identity" || data.kind === "account") {
          return;
        }

        console.log(`Received: ${JSON.stringify(data, null, 2)}`);
      } catch (e) {
        console.log(`Raw message: ${event.data}`);
      }
    };

    ws.onerror = (error) => {
      console.log(`WebSocket Error: ${error}`);
    };

    ws.onclose = () => {
      console.log("Disconnected from Bluesky WebSocket");
      // ws = null;
    };
  } catch (error) {
    console.log(`Connection Error: ${error}`);
    // ws = null;
  }
};

Here's @skyware/jetstream version

In [ ]:
//| export

import { Jetstream } from "@skyware/jetstream";
// import fs from "node:fs";

const jetstream = new Jetstream({
  wantedCollections: ["xyz.statusphere.*"],
  endpoint: "wss://jetstream2.us-east.bsky.network/subscribe",
  // cursor: Number(cursorFile),
});

type JetstreamClenup = () => void;

export function startJetstream(
  { onMessage }: { onMessage: (m: string) => void },
): JetstreamClenup {
  console.log("Starting jetstream");

  // let intervalID: NodeJS.Timeout;
  // const cursorFile = fs.readFileSync("cursor.txt", "utf8");
  // if (cursorFile) ctx.logger.info(`Initiate jetstream at cursor ${cursorFile}`);

  jetstream.on("error", (err) => console.error(err));
  // jetstream.on("close", () => clearInterval(intervalID));

  jetstream.on("open", () => {
    // intervalID = setInterval(() => {
    //   if (jetstream.cursor) {
    //     fs.writeFile("cursor.txt", jetstream.cursor.toString(), (err) => {
    //       if (err) console.log(err);
    //     });
    //   }
    // }, 60000);
  });

  jetstream.on("xyz.statusphere.status", (event) => {
    try {
      // if (event.commit.operation === "delete") await deleteProfile(event.did);
      // else await updateUser({ did: event.did, profile: event.commit.record });
      console.log(">>>>>>>>>>>>>>>", event);
      onMessage(JSON.stringify(event));
    } catch (err) {
      console.error(err, JSON.stringify(event));
    }
  });

  jetstream.start();
  console.log("Jetstream started");

  return () => {
    console.log("Stopping jetstream");
    jetstream.removeAllListeners();
    jetstream.close();
    console.log("Jetstream stopped");
  };
}

In [ ]:
import { assert } from "asserts";

Deno.test("jetstream start and stop", async (t) => {
  const cleanup = startJetstream({ onMessage: (m) => console.log(m) });

  await t.step("passing assertion", () => {
    assert(true);
  });

  cleanup();
});